In [6]:
%reload_ext autoreload
%autoreload 2

In [12]:
from openai import OpenAI
from pymilvus import connections, Collection

from elections import constants
from elections.ddl.articles_milvus_ddl import ARTICLES_COLLECTION

In [13]:
client = OpenAI()

# Connect to Milvus Database
connections.connect(host=constants.MILVUS_HOST, port=constants.MIVLUS_PORT)

In [9]:
import sqlite3
import pandas as pd
from elections import constants

engine = sqlite3.connect(constants.NEWS_DB)
df = pd.read_sql("SELECT * FROM articles LIMIT 5", engine)
df

,title,description,pubdate,publisher,url,summary,keywords,text,creation_datetime,query
0,Campanha a todo o gás e a prometedora entrevis...,Campanha a todo o gás e a prometedora entrevis...,2023-12-01T08:00:00,SIC Notícias,https://news.google.com/rss/articles/CBMilwFod...,"Para Luís Montenegro, é ""indiferente"" quem suc...","[""costa"", ""sic"", ""luís"", ""prometedora"", ""antón...","Para Luís Montenegro, é ""indiferente"" quem suc...",2024-02-11T17:26:45.678832,'Luis Montenegro'
1,"""Estamos mais preparados do que quaisquer outr...","""Estamos mais preparados do que quaisquer outr...",2023-12-02T08:00:00,Expresso,https://news.google.com/rss/articles/CBMijwFod...,O presidente do PSD defendeu este sábado que o...,"[""governar"", ""quaisquer"", ""programa"", ""preside...",O presidente do PSD defendeu este sábado que o...,2024-02-11T17:26:45.678832,'Luis Montenegro'
2,Luís Montenegro: “Qualquer pessoa tem o direit...,Luís Montenegro: “Qualquer pessoa tem o direit...,2024-02-10T15:30:00,Expresso,https://news.google.com/rss/articles/CBMiwgFod...,O cabeça de lista por Lisboa da Aliança Democr...,"[""sá"", ""tentou"", ""utilizar"", ""sic"", ""socialdem...",O cabeça de lista por Lisboa da Aliança Democr...,2024-02-11T17:40:25.902188,'Luis Montenegro'
3,Luís Montenegro nega “falta de comparência do ...,Luís Montenegro nega “falta de comparência do ...,2024-02-03T08:15:00,SIC Notícias,https://news.google.com/rss/articles/CBMioQFod...,"Para já, Luís Montenegro enfrenta a ameaça do ...","[""política"", ""passos"", ""casa"", ""espinho"", ""luí...",Aos 9 anos ia a pé para a escola e já “tinha a...,2024-02-11T17:40:25.902188,'Luis Montenegro'
4,Luís Montenegro recusa ter medo de debates pol...,Luís Montenegro recusa ter medo de debates pol...,2024-02-08T19:55:48,Público,https://news.google.com/rss/articles/CBMiaGh0d...,Líder social-democrata não responde a repto de...,"[""medo"", ""luís"", ""políticos"", ""ad"", ""debate"", ...",Líder social-democrata não responde a repto de...,2024-02-11T17:40:25.902188,'Luis Montenegro'


In [14]:
def create_embeddings(texts):
    embeddings = client.embeddings.create(
        input=texts,
        model=constants.OPENAI_EMBEDDING_MODEL
    )
    return [emb.embedding for emb in embeddings.data]


article_embedding = create_embeddings(df["text"])

In [33]:
collection = Collection(name=ARTICLES_COLLECTION)
collection.insert(df[["title", "description", "text", "embedding"]])

(insert count: 5, delete count: 0, upsert count: 0, timestamp: 447727778622078979, success count: 5, err count: 0)

In [ ]:
queries = [""]

In [37]:
for row in df.text:
    print(row)
    print(50 * "=")

Para Luís Montenegro, é "indiferente" quem sucederá a António Costa na liderança do Partido Socialista, mas não para Pedro Delgado Alves e José Eduardo Martins, neste Antes Pelo Contrário em podcast. Em entrevista à SIC, o líder do PSD lembrou que tanto José Luís Carneiro como Pedro Nuno Santos "fizeram parte dos governos de António Costa" que, segundo a sua avaliação, tiveram "muito pouco investimento público". Mas é ao antigo ministro das Infraestruturas que dirigiu o maior ataque: "Está associado a três das maiores trapalhadas que este governo trouxe para a política portuguesa". O Antes Pelo Contrário foi emitido a 30 de novembro na SIC Notícias.
O presidente do PSD defendeu este sábado que o seu partido é o mais bem preparado para governar Portugal, salientando que "não esteve à espera das eleições" e que tem já uma equipa de qualidade. Num encontro alargado do Conselho Estratégico Nacional (CEN) do PSD, que decorreu em Lisboa, Luís Montenegro salientou que este trabalho de prepara

Index(['title', 'description', 'pubdate', 'publisher', 'url', 'summary',
       'keywords', 'text', 'creation_datetime', 'query'],
      dtype='object')